## Install Libraries

In [1]:
_ = !pip install geopy pycountry-convert tqdm -q

## Import libraries

In [2]:
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pycountry_convert as pc

from pprint import pprint
from typing import Tuple

from tqdm import tqdm
tqdm.pandas()

In [3]:
site_df = pd.read_csv("https://gist.githubusercontent.com/benthecoder/7edb951977fdc5e80ad6a75ffb1626a1/raw/fb2f3a7a72b0dcfb2d137d691e5b64a3169281a3/site_metadata.csv")
site_df.head()

,CUSTOMER_NAME,PLANT_NAME,LATITUDE,LONGITUDE,ELEVATION,FUEL_N2_MOL_PCT,FUEL_MW,FUEL_LHV,CO2_FUEL_RATIO
0,SPIFFY,SPIRITUAL-POLECAT,61.170356,42.874767,112.000000,4.445063,16.572225,21514.222365,2.621803
1,NONCHALANT,NIFTY-ROOK,37.554515,49.908217,-29.000000,1.053145,16.166097,21526.470829,2.714870
2,NONCHALANT,PREHISTORIC-PETREL,29.190866,60.491702,1552.426025,10.298848,17.273122,21494.438672,2.461189
3,NONCHALANT,THERAPEUTIC-LIONFISH,13.253365,76.411056,867.591553,13.188814,17.619149,21485.251979,2.381895
4,SOFT,ABORIGINAL-PICULET,-68.632002,66.155301,1253.152832,7.581916,16.947813,21503.417436,2.535736


In [4]:
site_df[["LATITUDE", "LONGITUDE"]].values[0]

array([61.17035566, 42.87476723])

## Geopy geolocator

In [5]:
locator = Nominatim(user_agent="<APP_NAME>")
location = locator.reverse("61.170356,42.874767", language="en")
location

Location(Октябрьское городское поселение, Ustyansky District, Arkhangelsk Oblast, Northwestern Federal District, Russia, (61.2195686, 43.11476973933877, 0.0))

In [6]:
location.raw

{'place_id': 298376071,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'relation',
 'osm_id': 2062004,
 'lat': '61.2195686',
 'lon': '43.11476973933877',
 'display_name': 'Октябрьское городское поселение, Ustyansky District, Arkhangelsk Oblast, Northwestern Federal District, Russia',
 'address': {'municipality': 'Октябрьское городское поселение',
  'county': 'Ustyansky District',
  'state': 'Arkhangelsk Oblast',
  'ISO3166-2-lvl4': 'RU-ARK',
  'region': 'Northwestern Federal District',
  'country': 'Russia',
  'country_code': 'ru'},
 'boundingbox': ['61.0517979', '61.3870254', '42.7976429', '43.5165438']}

In [7]:
pprint(location.raw["address"])

{'ISO3166-2-lvl4': 'RU-ARK',
 'country': 'Russia',
 'country_code': 'ru',
 'county': 'Ustyansky District',
 'municipality': 'Октябрьское городское поселение',
 'region': 'Northwestern Federal District',
 'state': 'Arkhangelsk Oblast'}


## pycountry convert

In [8]:
pc.country_alpha2_to_continent_code("RU") # only uppercase

'EU'

In [1]:
def get_continent_name(continent_code: str) -> str:
    continent_dict = {
        "NA": "North America",
        "SA": "South America",
        "AS": "Asia",
        "AF": "Africa",
        "OC": "Oceania",
        "EU": "Europe",
        "AQ" : "Antarctica"
    }
    return continent_dict[continent_code]

get_continent_name("EU")

'Europe'

In [9]:
def get_continent(lat: float, lon:float) -> Tuple[str, str]:
    geolocator = Nominatim(user_agent="<username>@gmail.com", timeout=10)
    geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1)

    location = geocode(f"{lat}, {lon}", language="en")

    # for cases where the location is not found, coordinates are antarctica
    if location is None:
        return "Antarctica", "Antarctica"

    # extract country code
    address = location.raw["address"]
    country_code = address["country_code"].upper()

    # get continent code from country code
    continent_code = pc.country_alpha2_to_continent_code(country_code)
    continent_name = get_continent_name(continent_code)
    
    return country_name, continent_name

## Get location data

In [10]:
site_df[["COUNTRY", "CONTINENT"]] = site_df.progress_apply(
    lambda x: get_continent(x["LATITUDE"], x["LONGITUDE"]), axis=1, result_type="expand"
)

100%|██████████| 45/45 [00:22<00:00,  2.00it/s]


In [11]:
site_df.head()

,CUSTOMER_NAME,PLANT_NAME,LATITUDE,LONGITUDE,ELEVATION,FUEL_N2_MOL_PCT,FUEL_MW,FUEL_LHV,CO2_FUEL_RATIO,COUNTRY,CONTINENT
0,SPIFFY,SPIRITUAL-POLECAT,61.170356,42.874767,112.000000,4.445063,16.572225,21514.222365,2.621803,Russia,Europe
1,NONCHALANT,NIFTY-ROOK,37.554515,49.908217,-29.000000,1.053145,16.166097,21526.470829,2.714870,Iran,Asia
2,NONCHALANT,PREHISTORIC-PETREL,29.190866,60.491702,1552.426025,10.298848,17.273122,21494.438672,2.461189,Iran,Asia
3,NONCHALANT,THERAPEUTIC-LIONFISH,13.253365,76.411056,867.591553,13.188814,17.619149,21485.251979,2.381895,India,Asia
4,SOFT,ABORIGINAL-PICULET,-68.632002,66.155301,1253.152832,7.581916,16.947813,21503.417436,2.535736,Antarctica,Antarctica


## Plot coordinates

In [12]:
import plotly.express as px

fig = px.scatter_geo(
    site_df,
    lat="LATITUDE",
    lon="LONGITUDE",
    color="CONTINENT",
    hover_data=["COUNTRY"],
    projection="natural earth",
    size="FUEL_N2_MOL_PCT",
    width=800,
    height=600,
)

# set dark theme
fig.update_layout(
    # template="plotly_dark",
    coloraxis_showscale=False,
    margin=dict(l=0, r=0, t=40, b=0),
)
fig.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a605a3e6-1564-47b2-94e7-842290ba7692' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>